In [1]:
import os
#import h5py
#import json
import numpy as np
import random
#import scipy.io.wavfile
#from scipy.io.wavfile import read
#from scipy import signal
#from sklearn.decomposition import NMF

d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#f = open('./imagenet1000_clsid_to_human.json', encoding='utf-8')
#label = json.load(f)
#label = dict(zip(label.values(), label.keys()))
indexes1 = ['accordion', 'acoustic_guitar', 'banjo', 'cello', 'drum', 'electric_guitar', 'flute', 'french_horn', 'harmonica', 'harp', 'marimba', 'piano', 'saxophone', 'trombone', 'violin']
indexes2 = [[401], [402], [420], [486], [541], [546], [558], [566], [593], [594], [642], [579,881], [776], [875], [889]]
#indexes2 = [[401], [402], [420], [486], [541], [546], [558], [566], [593], [594], [642], [579], [776], [875], [889]]
#label = { k:v[0] for k,v in zip(indexes1,indexes2)}
#print(label)

## preprocess dataset
1. compute labels of images using ResNet101, and save the results as `*_labels.npy` files
2. compute NMF and save the result as `*_bases.npy` files
3. filepaths of `*_labels.npy`&`*_bases.npy` are stored in `train.h5`,`val.h5` and `test.h5`

In [3]:
# Use pre-trained weights for Tensorflow backend
weights_path = 'ResNet/weights/ResNet-101-model.keras.h5'  #weights_path = 'weights/resnet101_weights_tf.h5'

# Test pretrained model - Inference mode
model = resnet101_model(weights_path)

In [4]:
#from dataset.NMF import *
from dataset.NMF import audio_import
from ResNet import resnet101_keras
from ResNet.resnet101_keras import resnet101_model, resnet_predict

path = 'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/'
dir_list = os.listdir(path)
numOfTrain = 0
bases = []
labels = []
flag=True

for cur_dir in dir_list:
    #忽略非乐器数据
    if cur_dir not in indexes1:
        continue
        
    #读取wav文件
    for f in os.listdir(path+cur_dir):
        if '.wav' in f:
            fname = path+cur_dir+'/'+f
            break
    print([fname])
    
    #计算NMF并保存.npy文件
    if flag:
        nmf_base = audio_import.nmf([fname])[0]  #每个文件夹下只有一个wav文件
        print(nmf_base.shape)
        #flag=False
    np.save(path+cur_dir+'/'+f[5:-4]+'_bases'+'.npy', nmf_base)
    bases.append(path+cur_dir+'/'+f[5:-4]+'_bases'+'.npy')
    
    #读取当前文件夹下所有图片文件
    img_files = []
    #print(path+cur_dir)
    for root,dirs,files in os.walk(path+cur_dir, topdown=False):
        #print(root)
        for name in files:
            if '.png' in name or '.jpg' in name:
                img_files.append(os.path.normcase(root+os.path.sep+name).replace('\\','/'))
                #print(img_file[-1])
    #print(img_files)
    
    #用ResNet101检测图像中物体种类
    softmax = np.zeros(1000)
    for img in random.sample(img_files, len(img_files)//3):  #为了节约时间，随机抽取1/3的图片进行预测
        softmax = softmax + resnet_predict(model, img)[0]
    np.save(path+cur_dir+'/'+f[5:-4]+'_labels'+'.npy', softmax)
    labels.append(path+cur_dir+'/'+f[5:-4]+'_labels'+'.npy')
    
    numOfTrain = numOfTrain+1
    
    '''
    sample_rate, samples = scipy.io.wavfile.read(fname)
    frequencies, times, spectrogram = signal.spectrogram(samples, fs=sample_rate, nperseg=4410, noverlap=nperseg//2, nfft=2401)
    #print(sr_value)
    #print(x_value)'''
    
print(bases)
print(labels)

['e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/accordion/audioAareFwTIg1s.wav']
(2401, 25)
image size: (224, 224, 3)
batch_shape (1, 224, 224, 3)
softmax shape: (1, 1000)
Prediction ID: 401
Class: accordion, piano accordion, squeeze box
Score: 0.99976915

image size: (224, 224, 3)
batch_shape (1, 224, 224, 3)
softmax shape: (1, 1000)
Prediction ID: 401
Class: accordion, piano accordion, squeeze box
Score: 0.99918014

image size: (224, 224, 3)
batch_shape (1, 224, 224, 3)
softmax shape: (1, 1000)
Prediction ID: 401
Class: accordion, piano accordion, squeeze box
Score: 0.99933404

['e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/acoustic_guitar/audio2K61zdPjAi8.wav']
(2401, 25)
image size: (224, 224, 3)
batch_shape (1, 224, 224, 3)
softmax shape: (1, 1000)
Prediction ID: 402
Class: acoustic guitar
Score: 0.79531056

image size: (224, 224, 3)
batch_shape (1, 224, 224, 3)
softmax shape: (1, 1000)
Prediction ID: 402
Class: acoustic guitar


## generate `train.h5` and `val.h5`

In [11]:
import h5py

bases_encode = []
labels_encode = []
for i,j in zip(bases,labels):
    bases_encode.append(i.encode(encoding='utf-8', errors='strict'))
    labels_encode.append(j.encode(encoding='utf-8', errors='strict'))


h5f = h5py.File('dataset/train.h5', 'w')
h5f.create_dataset('bases', data=bases_encode[0:numOfTrain])
h5f.create_dataset('labels', data=labels_encode[0:numOfTrain])
h5f.close()

h5f = h5py.File('dataset/val.h5', 'w')
h5f.create_dataset('bases', data=bases_encode[0:numOfTrain//2])
h5f.create_dataset('labels', data=labels_encode[0:numOfTrain//2])
h5f.close()

## generate `test.h5`

In [2]:
import h5py

h5f = h5py.File('dataset/train.h5', 'r')
test_bases = []
test_labels = []

for name in h5f:
    print(name)
    if name == 'bases':
        for v in h5f[name]:
            test_bases.append(v)
    if name == 'labels':
        for v in h5f[name]:
            test_labels.append(v)
    
h5f.close()

#print(test_bases)
#print(test_labels)

h5f = h5py.File('dataset/test.h5', 'w')
h5f.create_dataset('bases', data=test_bases[-3:])
h5f.create_dataset('labels', data=test_labels[-3:])
h5f.close()

bases
labels
[b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/accordion/AareFwTIg1s_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/acoustic_guitar/2K61zdPjAi8_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/banjo/PpJKo-JPVU0_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/cello/B1ixRtiUJ-U_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/drum/b98BJ36K1wo_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/electric_guitar/2BBAUwC1BI4_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/flute/_5w5TVK5B90_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/french_horn/chwqnnaiYXw_bases.npy', b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/harmonica/5rjQU4vOIlw_bases.npy', b'e:/Study/Vision

## read data in `train.h5`

In [12]:
h5f = h5py.File('dataset/train.h5', 'r')
for name in h5f:
    print(name)
    for v in h5f[name]:
        print(v)
    
h5f.close()

bases
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/accordion/AareFwTIg1s_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/acoustic_guitar/2K61zdPjAi8_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/banjo/PpJKo-JPVU0_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/cello/B1ixRtiUJ-U_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/drum/b98BJ36K1wo_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/electric_guitar/2BBAUwC1BI4_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/flute/_5w5TVK5B90_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/french_horn/chwqnnaiYXw_bases.npy'
b'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/harmonica/5rjQU4vOIlw_bases.npy'
b'e:/Study/Vision/project/Deep-MIM

In [10]:
h5f.close()

In [2]:
softmax = resnet_predict('ResNet/images/000001.jpg')

image size: (224, 224, 3)
batch_shape (1, 224, 224, 3)
softmax shape: (1, 1000)
Prediction ID: 401
Class: accordion, piano accordion, squeeze box
Score: 0.9954704



In [5]:
print(softmax.shape)

(1, 1000)
